In [1]:
from pathlib import Path
import pandas as pd

In [2]:
path_ds_training = Path("./OIA-ODIR/Training Set/Annotation/training annotation (English).xlsx")
path_ds_training_img = Path("./OIA-ODIR/Training Set/Images")
path_ds_validation = Path("./OIA-ODIR/Off-site Test Set/Annotation/off-site test annotation (English).xlsx")
path_ds_validation_img = Path("./OIA-ODIR/Off-site Test Set/Images")
path_ds_test = Path("./OIA-ODIR/On-site Test Set/Annotation/on-site test annotation (English).xlsx")
path_ds_test_img = Path("./OIA-ODIR/On-site Test Set/Images")

ds_training = pd.read_excel(path_ds_training, index_col=0)
ds_validation = pd.read_excel(path_ds_validation, index_col=0)
ds_test = pd.read_excel(path_ds_test, index_col=0)

### Ajustando o nome das colunas

In [3]:
columns = ["age", "sex", "left_file", "right_file", "left_keywords", "right_keywords", "N", "D", "G", "C", "A","H", "M", "O" ]
ds_training.columns = columns
ds_validation.columns = columns
ds_test.columns = columns

#### Excluindo registro com imagens ruins

In [4]:
# removendo imagens sem qualidade
to_drop = ds_training.query("left_file in (\"2174_left.jpg\", \"2175_left.jpg\", \"2176_left.jpg\", \"2177_left.jpg\", \"2178_left.jpg\", \"2180_left.jpg\", \"2181_left.jpg\", \"2182_left.jpg\", \"2957_left.jpg\")")
ds_training.drop(to_drop.index, inplace = True)

to_drop = ds_validation.query("left_file in (\"2174_left.jpg\", \"2175_left.jpg\", \"2176_left.jpg\", \"2177_left.jpg\", \"2178_left.jpg\", \"2180_left.jpg\", \"2181_left.jpg\", \"2182_left.jpg\", \"2957_left.jpg\")")
ds_validation.drop(to_drop.index, inplace = True)

to_drop = ds_test.query("left_file in (\"2174_left.jpg\", \"2175_left.jpg\", \"2176_left.jpg\", \"2177_left.jpg\", \"2178_left.jpg\", \"2180_left.jpg\", \"2181_left.jpg\", \"2182_left.jpg\", \"2957_left.jpg\")")
ds_test.drop(to_drop.index, inplace = True)


# Ajustando o dataset por olho

Na exporação do dataset descrobrimos que as classificações não estão por imagens e sim por Paciente, então vamos reorganizar o dataset por imagem e reclassifica-las
Depois vamos buscar imagens que foram diagnosticadas apenas com 1 doença e vamos desconsiderar imagens com mais de 1 classificação. Assim buscamos melhorar a performance da Rede.


### Metodo auxiliar para separar cada olho em uma linha

In [7]:
def get_ds_eyes_line(ds):
    left_features = ["left_file", "left_keywords" ]
    left_ds_eye = ds[left_features]
    left_ds_eye.insert(1, "side","left")

    left_ds_eye = left_ds_eye.rename(columns={
                                                "left_file": "file_name", 
                                                "left_keywords":"keywords"  })


    right_features = ["right_file", "right_keywords"]
    right_ds_eye = ds[right_features]
    right_ds_eye.insert(1, "side","right")


    right_ds_eye = right_ds_eye.rename(columns={"right_file": "file_name", 
                                                            "right_keywords":"keywords"})

    ds_eye = pd.concat([left_ds_eye, right_ds_eye], ignore_index=True)
    ds_eye =  ds_eye.sort_values(by=["file_name"],ignore_index=True)
    return ds_eye


In [8]:
ds_training_eye = get_ds_eyes_line(ds_training)
ds_validation_eye = get_ds_eyes_line(ds_validation)
ds_test_eye = get_ds_eyes_line(ds_test)


In [9]:
ds_training_eye.head()

,file_name,side,keywords
0,0_left.jpg,left,cataract
1,0_right.jpg,right,normal fundus
2,1005_left.jpg,left,macular hole
3,1005_right.jpg,right,normal fundus
4,1006_left.jpg,left,normal fundus


### Metodos auxiliares para a reclassificação das imagens

In [10]:
# Metodos criados para reclassificar os imagens do Dataset

separador = ","

def ajusta_separador(keywords):
    return keywords.replace("，", ",").replace(",",",")

def classifica_desconsiderar(keywords):
    
    keys = keywords.split(separador)
    
    for key in keys:
        key = key.strip()
        #key == "lens dust" or key == "optic disk photographically invisible"
        if key == "low image quality" or key == "image offset" or key == "mild nonproliferative retinopathy" or key == "mild non proliferative retinopathy":
            return 1
    
    return 0

def classifica_normal(keywords):
    keys = keywords.split(separador)
    
    for key in keys:
        key = key.strip()
        if key == "normal fundus":
            return 1
    
    return 0

def classifica_diabetes(keywords):
    keys = keywords.split(separador)

    for key in keys:
        key = key.strip()
                   
        if key == "moderate non proliferative retinopathy" or key == "moderate nonproliferative retinopathy" or key == "diabetic retinopathy" or key == "proliferative diabetic retinopathy" or key == "severe nonproliferative retinopathy" or key == "severe proliferative diabetic retinopathy":
            return 1
    
    return 0

def classifica_glaucoma(keywords):
    keys = keywords.split(separador)
    
    for key in keys:
        key = key.strip()
        if key == "glaucoma" or key == "suspected glaucoma":
            return 1
    
    return 0

def classifica_cataract(keywords):
    keys = keywords.split(separador)
    
    for key in keys:
        key = key.strip()
        if key == "cataract":
            return 1
    
    return 0

def classifica_degeneracao_macular(keywords):
    keys = keywords.split(separador)
    
    for key in keys:
        key = key.strip()
        if key == "dry age-related macular degeneration" or  key == "wet age-related macular degeneration" or key == "age-related macular degeneration":
            return 1
    
    return 0

def  classifica_pressao_alta(keywords):
    keys = keywords.split(separador)
    
    for key in keys:
        key = key.strip()
        if key == "hypertensive retinopathy":
            return 1
    
    return 0

def  classifica_miopia_patologica(keywords):
    keys = keywords.split(separador)
    
    for key in keys:
        key = key.strip()
        if key == "pathological myopia":
            return 1
    
    return 0

def classifica_outras(keywords):
    """
        N - Normal
        D - Diabetes
        G - Glaucoma
        C - Catarata
        A - Degeneração macular
        H - Pressão alta
        M - Miopia patológica 
        O - Outros
    """
    
    
    keys = keywords.split(separador)
    for key in keys:
        key = key.strip()
        if key == "normal fundus":
            continue
    
        if key == "moderate non proliferative retinopathy" or key == "moderate nonproliferative retinopathy" or key == "mild nonproliferative retinopathy" or key == "mild non proliferative retinopathy" or key == "diabetic retinopathy" or key == "proliferative diabetic retinopathy" or key == "severe nonproliferative retinopathy" or key == "severe proliferative diabetic retinopathy":
            continue

        if key == "glaucoma" or key == "suspected glaucoma":
            continue

        if key == "cataract":
            continue

        if key == "dry age-related macular degeneration" or  key == "wet age-related macular degeneration" or key == "age-related macular degeneration":
            continue

        if key == "hypertensive retinopathy":
            continue

        if key == "pathological myopia":
            continue
        
        if key == "anterior segment image" or key == "no fundus image" or key == "lens dust" or key == "optic disk photographically invisible" or key ==  "low image quality":
            continue
        
        return 1
    
    return 0

In [11]:
def class_ds(ds):
    
    ds["keywords"] = ds["keywords"].map(ajusta_separador)

    ds["N"] = ds["keywords"].map(classifica_normal)
    ds["D"] = ds["keywords"].map(classifica_diabetes)
    ds["G"] = ds["keywords"].map(classifica_glaucoma)
    ds["C"] = ds["keywords"].map(classifica_cataract)
    ds["A"] = ds["keywords"].map(classifica_degeneracao_macular)
    ds["H"] = ds["keywords"].map(classifica_pressao_alta)
    ds["M"] = ds["keywords"].map(classifica_miopia_patologica)
    ds["O"] = ds["keywords"].map(classifica_outras)
    ds["X"] = ds["keywords"].map(classifica_desconsiderar)
    
    #return ds;

In [12]:
class_ds(ds_training_eye)
class_ds(ds_validation_eye)
class_ds(ds_test_eye)

#class_ds(ds_all_eye)


In [13]:
ds_training_eye.head()

,file_name,side,keywords,N,D,G,C,A,H,M,O,X
0,0_left.jpg,left,cataract,0,0,0,1,0,0,0,0,0
1,0_right.jpg,right,normal fundus,1,0,0,0,0,0,0,0,0
2,1005_left.jpg,left,macular hole,0,0,0,0,0,0,0,1,0
3,1005_right.jpg,right,normal fundus,1,0,0,0,0,0,0,0,0
4,1006_left.jpg,left,normal fundus,1,0,0,0,0,0,0,0,0


In [14]:
print(ds_training_eye.query("N == 1").file_name.count())
print(ds_training_eye.query("D == 1").file_name.count())
print(ds_training_eye.query("G == 1").file_name.count())
print(ds_training_eye.query("C == 1").file_name.count())
print(ds_training_eye.query("A == 1").file_name.count())
print(ds_training_eye.query("H == 1").file_name.count())
print(ds_training_eye.query("M == 1").file_name.count())
print(ds_training_eye.query("O == 1").file_name.count())



3093
1247
326
300
280
193
246
1227


In [15]:
ds_training_eye["total"]  = ds_training_eye.N +  ds_training_eye.D + ds_training_eye.G + ds_training_eye.C + ds_training_eye.A + ds_training_eye.H + ds_training_eye.M + ds_training_eye.O + ds_training_eye.X
ds_validation_eye["total"]  = ds_validation_eye.N +  ds_validation_eye.D + ds_validation_eye.G + ds_validation_eye.C + ds_validation_eye.A + ds_validation_eye.H + ds_validation_eye.M + ds_validation_eye.O + ds_validation_eye.X
ds_test_eye["total"]  = ds_test_eye.N +  ds_test_eye.D + ds_test_eye.G + ds_test_eye.C + ds_test_eye.A + ds_test_eye.H + ds_test_eye.M + ds_test_eye.O + ds_test_eye.X

#ds_all_eye["total"] = ds_all_eye.N +  ds_all_eye.D + ds_all_eye.G + ds_all_eye.C + ds_all_eye.A + ds_all_eye.H + ds_all_eye.M + ds_all_eye.O + ds_all_eye.X

In [16]:
ds_training_eye



,file_name,side,keywords,N,D,G,C,A,H,M,O,X,total
0,0_left.jpg,left,cataract,0,0,0,1,0,0,0,0,0,1
1,0_right.jpg,right,normal fundus,1,0,0,0,0,0,0,0,0,1
2,1005_left.jpg,left,macular hole,0,0,0,0,0,0,0,1,0,1
3,1005_right.jpg,right,normal fundus,1,0,0,0,0,0,0,0,0,1
4,1006_left.jpg,left,normal fundus,1,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6977,999_right.jpg,right,cataract,0,0,0,1,0,0,0,0,0,1
6978,99_left.jpg,left,macular epiretinal membrane,0,0,0,0,0,0,0,1,0,1
6979,99_right.jpg,right,normal fundus,1,0,0,0,0,0,0,0,0,1
6980,9_left.jpg,left,normal fundus,1,0,0,0,0,0,0,0,0,1


In [17]:
ds_training_eye.query("O == 1 and total == 1")


,file_name,side,keywords,N,D,G,C,A,H,M,O,X,total
2,1005_left.jpg,left,macular hole,0,0,0,0,0,0,0,1,0,1
5,1006_right.jpg,right,macular epiretinal membrane,0,0,0,0,0,0,0,1,0,1
7,1008_right.jpg,right,epiretinal membrane,0,0,0,0,0,0,0,1,0,1
8,100_left.jpg,left,macular epiretinal membrane,0,0,0,0,0,0,0,1,0,1
11,1010_right.jpg,right,drusen,0,0,0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6961,990_right.jpg,right,macular epiretinal membrane,0,0,0,0,0,0,0,1,0,1
6969,994_right.jpg,right,chorioretinal atrophy,0,0,0,0,0,0,0,1,0,1
6970,996_left.jpg,left,myelinated nerve fibers,0,0,0,0,0,0,0,1,0,1
6978,99_left.jpg,left,macular epiretinal membrane,0,0,0,0,0,0,0,1,0,1


In [18]:
ds_training_eye.query("file_name == '1467_left.jpg'")

,file_name,side,keywords,N,D,G,C,A,H,M,O,X,total


In [19]:
ds_training_eye.query("O == 1")

,file_name,side,keywords,N,D,G,C,A,H,M,O,X,total
2,1005_left.jpg,left,macular hole,0,0,0,0,0,0,0,1,0,1
5,1006_right.jpg,right,macular epiretinal membrane,0,0,0,0,0,0,0,1,0,1
7,1008_right.jpg,right,epiretinal membrane,0,0,0,0,0,0,0,1,0,1
8,100_left.jpg,left,macular epiretinal membrane,0,0,0,0,0,0,0,1,0,1
11,1010_right.jpg,right,drusen,0,0,0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6970,996_left.jpg,left,myelinated nerve fibers,0,0,0,0,0,0,0,1,0,1
6972,997_left.jpg,left,"diabetic retinopathy,post laser photocoagulation",0,1,0,0,0,0,0,1,0,2
6973,997_right.jpg,right,"diabetic retinopathy,post laser photocoagulation",0,1,0,0,0,0,0,1,0,2
6978,99_left.jpg,left,macular epiretinal membrane,0,0,0,0,0,0,0,1,0,1


### Imagens abaixo serão desconsideradas
<p>Todas as imagens classificadas com mais de uma doença será removida do dataset</p>

In [20]:
print("Dataset treinamento")
print("Total com mais de 1 diagnostico:", ds_training_eye.query("total > 1").side.count())
print(ds_training_eye.query("total > 1 and D == 1").side.count())
print(ds_training_eye.query("total > 1 and G == 1").side.count())
print(ds_training_eye.query("total > 1 and C == 1").side.count())
print(ds_training_eye.query("total > 1 and A == 1").side.count())
print(ds_training_eye.query("total > 1 and H == 1").side.count())
print(ds_training_eye.query("total > 1 and M == 1").side.count())
print(ds_training_eye.query("total > 1 and O == 1").side.count())
print(ds_training_eye.query("total > 1 and X == 1").side.count())

print("Dataset validação")
print("Total com mais de 1 diagnostico:", ds_training_eye.query("total > 1").side.count())
print(ds_validation_eye.query("total > 1 and D == 1").side.count())
print(ds_validation_eye.query("total > 1 and G == 1").side.count())
print(ds_validation_eye.query("total > 1 and C == 1").side.count())
print(ds_validation_eye.query("total > 1 and A == 1").side.count())
print(ds_validation_eye.query("total > 1 and H == 1").side.count())
print(ds_validation_eye.query("total > 1 and M == 1").side.count())
print(ds_validation_eye.query("total > 1 and O == 1").side.count())
print(ds_validation_eye.query("total > 1 and X == 1").side.count())

print("Dataset teste")
print("Total com mais de 1 diagnostico:", ds_training_eye.query("total > 1").side.count())
print(ds_test_eye.query("total > 1 and D == 1").side.count())
print(ds_test_eye.query("total > 1 and G == 1").side.count())
print(ds_test_eye.query("total > 1 and C == 1").side.count())
print(ds_test_eye.query("total > 1 and A == 1").side.count())
print(ds_test_eye.query("total > 1 and H == 1").side.count())
print(ds_test_eye.query("total > 1 and M == 1").side.count())
print(ds_test_eye.query("total > 1 and O == 1").side.count())
print(ds_test_eye.query("total > 1 and X == 1").side.count())



Dataset treinamento
Total com mais de 1 diagnostico: 502
313
99
27
43
89
13
341
84
Dataset validação
Total com mais de 1 diagnostico: 502
34
13
2
8
15
2
45
15
Dataset teste
Total com mais de 1 diagnostico: 502
72
18
14
9
27
4
84
29


### Imagens com apenas 1 classificação
<p>Essa imagens serão utilizadas para o treinamento</p>

In [21]:
print("Dataset treinamento")
print("Total com apenas 1 diagnostico:", ds_training_eye.query("total == 1").side.count())
print(ds_training_eye.query("total == 1 and N == 1").side.count())
print(ds_training_eye.query("total == 1 and D == 1").side.count())
print(ds_training_eye.query("total == 1 and G == 1").side.count())
print(ds_training_eye.query("total == 1 and C == 1").side.count())
print(ds_training_eye.query("total == 1 and A == 1").side.count())
print(ds_training_eye.query("total == 1 and H == 1").side.count())
print(ds_training_eye.query("total == 1 and M == 1").side.count())
print(ds_training_eye.query("total == 1 and O == 1").side.count())
print(ds_training_eye.query("total == 1 and X == 1").side.count())

print("Dataset validação")
print("Total com apenas 1 diagnostico:", ds_training_eye.query("total == 1").side.count())
print(ds_validation_eye.query("total == 1 and N == 1").side.count())
print(ds_validation_eye.query("total == 1 and D == 1").side.count())
print(ds_validation_eye.query("total == 1 and G == 1").side.count())
print(ds_validation_eye.query("total == 1 and C == 1").side.count())
print(ds_validation_eye.query("total == 1 and A == 1").side.count())
print(ds_validation_eye.query("total == 1 and H == 1").side.count())
print(ds_validation_eye.query("total == 1 and M == 1").side.count())
print(ds_validation_eye.query("total == 1 and O == 1").side.count())
print(ds_validation_eye.query("total == 1 and X == 1").side.count())

print("Dataset teste")
print("Totalcom apenas 1 diagnostico:", ds_training_eye.query("total == 1").side.count())
print(ds_test_eye.query("total == 1 and N == 1").side.count())
print(ds_test_eye.query("total == 1 and D == 1").side.count())
print(ds_test_eye.query("total == 1 and G == 1").side.count())
print(ds_test_eye.query("total == 1 and C == 1").side.count())
print(ds_test_eye.query("total == 1 and A == 1").side.count())
print(ds_test_eye.query("total == 1 and H == 1").side.count())
print(ds_test_eye.query("total == 1 and M == 1").side.count())
print(ds_test_eye.query("total == 1 and O == 1").side.count())
print(ds_test_eye.query("total == 1 and X == 1").side.count())


Dataset treinamento
Total com apenas 1 diagnostico: 6471
3090
934
227
273
237
104
233
886
487
Dataset validação
Total com apenas 1 diagnostico: 6471
430
126
32
45
36
11
37
130
80
Dataset teste
Totalcom apenas 1 diagnostico: 6471
819
276
65
81
70
27
48
346
117


In [22]:
ds_training_eye.query("total == 1 and N == 1")

,file_name,side,keywords,N,D,G,C,A,H,M,O,X,total
1,0_right.jpg,right,normal fundus,1,0,0,0,0,0,0,0,0,1
3,1005_right.jpg,right,normal fundus,1,0,0,0,0,0,0,0,0,1
4,1006_left.jpg,left,normal fundus,1,0,0,0,0,0,0,0,0,1
6,1008_left.jpg,left,normal fundus,1,0,0,0,0,0,0,0,0,1
9,100_right.jpg,right,normal fundus,1,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6960,990_left.jpg,left,normal fundus,1,0,0,0,0,0,0,0,0,1
6968,994_left.jpg,left,normal fundus,1,0,0,0,0,0,0,0,0,1
6971,996_right.jpg,right,normal fundus,1,0,0,0,0,0,0,0,0,1
6979,99_right.jpg,right,normal fundus,1,0,0,0,0,0,0,0,0,1


In [23]:
ds_test_eye.query("total == 1 and N == 1")

,file_name,side,keywords,N,D,G,C,A,H,M,O,X,total
17,1175_right.jpg,right,normal fundus,1,0,0,0,0,0,0,0,0,1
24,1187_left.jpg,left,normal fundus,1,0,0,0,0,0,0,0,0,1
37,1204_right.jpg,right,normal fundus,1,0,0,0,0,0,0,0,0,1
48,1313_left.jpg,left,normal fundus,1,0,0,0,0,0,0,0,0,1
50,1316_left.jpg,left,normal fundus,1,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1972,4987_left.jpg,left,normal fundus,1,0,0,0,0,0,0,0,0,1
1983,4992_right.jpg,right,normal fundus,1,0,0,0,0,0,0,0,0,1
1988,4995_left.jpg,left,normal fundus,1,0,0,0,0,0,0,0,0,1
1993,4997_right.jpg,right,normal fundus,1,0,0,0,0,0,0,0,0,1


### Criando um novo data
<p> Aqui vamos criar um dataset com imagens apenas 1 classificação e vamos descartar as doenças abaixo:</p>
    <ul>
    <li>Glaucoma - Não tem imagens suficientes para um bom treinamento</li>
    <li>Pressão alta - Precimos de imagens com definição melhor</li>
    <li>Outros - São diversar doenças com caracteristicas distintas</li>
    </ul>
    
    

In [24]:
ds_new_train = ds_training_eye.query("total == 1  and O == 0 and X == 0")
ds_new_valid = ds_validation_eye.query("total == 1 and O == 0 and X == 0")
ds_new_test = ds_test_eye.query("total == 1  and O == 0 and X == 0")

In [25]:
ds_new_train.head(15)

,file_name,side,keywords,N,D,G,C,A,H,M,O,X,total
0,0_left.jpg,left,cataract,0,0,0,1,0,0,0,0,0,1
1,0_right.jpg,right,normal fundus,1,0,0,0,0,0,0,0,0,1
3,1005_right.jpg,right,normal fundus,1,0,0,0,0,0,0,0,0,1
4,1006_left.jpg,left,normal fundus,1,0,0,0,0,0,0,0,0,1
6,1008_left.jpg,left,normal fundus,1,0,0,0,0,0,0,0,0,1
9,100_right.jpg,right,normal fundus,1,0,0,0,0,0,0,0,0,1
10,1010_left.jpg,left,normal fundus,1,0,0,0,0,0,0,0,0,1
12,1011_left.jpg,left,normal fundus,1,0,0,0,0,0,0,0,0,1
15,1012_right.jpg,right,pathological myopia,0,0,0,0,0,0,1,0,0,1
16,1013_left.jpg,left,normal fundus,1,0,0,0,0,0,0,0,0,1


In [26]:
ds_new_valid

,file_name,side,keywords,N,D,G,C,A,H,M,O,X,total
5,1002_right.jpg,right,moderate nonproliferative retinopathy,0,1,0,0,0,0,0,0,0,1
8,1004_left.jpg,left,moderate nonproliferative retinopathy,0,1,0,0,0,0,0,0,0,1
13,1009_right.jpg,right,moderate nonproliferative retinopathy,0,1,0,0,0,0,0,0,0,1
15,1026_right.jpg,right,moderate nonproliferative retinopathy,0,1,0,0,0,0,0,0,0,1
16,1027_left.jpg,left,moderate nonproliferative retinopathy,0,1,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
988,4791_left.jpg,left,severe nonproliferative retinopathy,0,1,0,0,0,0,0,0,0,1
989,4791_right.jpg,right,severe nonproliferative retinopathy,0,1,0,0,0,0,0,0,0,1
992,937_left.jpg,left,hypertensive retinopathy,0,0,0,0,0,1,0,0,0,1
994,967_left.jpg,left,moderate nonproliferative retinopathy,0,1,0,0,0,0,0,0,0,1


In [27]:
ds_new_test

,file_name,side,keywords,N,D,G,C,A,H,M,O,X,total
1,1025_right.jpg,right,hypertensive retinopathy,0,0,0,0,0,1,0,0,0,1
2,1139_left.jpg,left,moderate nonproliferative retinopathy,0,1,0,0,0,0,0,0,0,1
8,1155_left.jpg,left,moderate nonproliferative retinopathy,0,1,0,0,0,0,0,0,0,1
10,1158_left.jpg,left,moderate nonproliferative retinopathy,0,1,0,0,0,0,0,0,0,1
12,1159_left.jpg,left,moderate nonproliferative retinopathy,0,1,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1993,4997_right.jpg,right,normal fundus,1,0,0,0,0,0,0,0,0,1
1994,4998_left.jpg,left,normal fundus,1,0,0,0,0,0,0,0,0,1
1995,4998_right.jpg,right,moderate nonproliferative retinopathy,0,1,0,0,0,0,0,0,0,1
1996,4999_left.jpg,left,moderate nonproliferative retinopathy,0,1,0,0,0,0,0,0,0,1


### Salvando os novos dataset

In [28]:
ds_new_train.to_excel("training_final.xlsx") 
ds_new_valid.to_excel("validation_final.xlsx") 
ds_new_test.to_excel("test_final.xlsx") 